In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from collections import Counter

In [2]:
mainDF = pd.read_excel("Data//October_10//LabHoursData.xlsx")
mainDF.columns

Index(['#', 'Machine Type', 'Machine', 'Wing', 'Floor', 'Run No', 'Pcs',
       'StartDate', 'StartTime', 'StopDate', 'StopTime', 'Run %',
       'LateStartReason', 'Stop Reasons', 'Recipe', 'Run Intensity',
       'Ini Category', 'Grown Category', 'RunStatus', 'IniAvgHeight',
       'ProdHeight', 'GrownHeight', 'GrowingHours', 'BreakdownTime',
       'From To Hours', 'Actual Hours', 'RunHours', 'IniWeight', 'FinalWeight',
       'GrownWeight', 'Width', 'Length', 'Dmtr', 'Unit', 'Month', 'GrRate'],
      dtype='object')

In [3]:
#remove = "BreakdownTime", "GrowingHours"
final_DF = mainDF[["Machine Type", "Machine", "GrownWeight", "ProdHeight", "Dmtr", 
                   "Actual Hours", "GrRate", "Stop Reasons", "LateStartReason"]]
#final_DF[final_DF.isna().any(axis=1)]
final_DF = final_DF.rename(columns = {'Machine Type':'Machine_Type', 'Stop Reasons':'Stop_Reasons', 'Actual Hours':'Actual_Hours'})

final_DF.tail(2)

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
5303,New,AG42,59.228665,3.358553,41.13,192.0,12.14,"TIDE O , N/A",ON TIME
5304,NaN,NaN,NaN,NaN,NaN,NaN,Avg: 12.51,NaN,NaN


In [4]:
nullID = final_DF[final_DF[['Machine_Type','Machine','GrownWeight', 'ProdHeight','Dmtr', \
                            'Actual_Hours', 'GrRate']].isna().any(axis=1)].index
nullID

Int64Index([5304], dtype='int64')

In [5]:
final_DF = final_DF.drop(nullID)

In [6]:
final_DF.head(2)

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
0,Old,CA2,97.87,3.777692,50.71,180.0,13.67,"TIDE O , V LEAK",ON TIME
1,Old,CA2,76.91,4.452222,46.92,160.0,13.56,"TIDE O , V LEAK",ON TIME


In [7]:
def count_words(df_temp,column_name):
    df_temp[column_name] = df_temp[column_name].str.rstrip('')
    df_temp[column_name] = df_temp[column_name].str.lstrip('')

    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(',,',',')

    df_temp[column_name] = df_temp[column_name].str.replace('\' ','\'')
    df_temp[column_name] = df_temp[column_name].str.replace(' \'','\'')

    df_temp[column_name] = df_temp[column_name].str.rstrip(',')
    df_temp[column_name] = df_temp[column_name].str.lstrip(',')

    df_temp[column_name] = df_temp[column_name].str.split(',')
    df_temp[column_name] = df_temp[column_name].apply(lambda x: Counter(x))
    
    df_temp[column_name] = df_temp[column_name].apply(lambda x: dict(x))
    
    return df_temp

In [8]:
#join all string values of Stop_Reasons in groupby
df_Stop_reason = final_DF.groupby(['Machine','Machine_Type'])['Stop_Reasons'].agg(lambda x: ','.join(x.dropna())).reset_index()

#join all string values of LateStartReason in groupby
df_LateStartReason = final_DF.groupby(['Machine','Machine_Type'])['LateStartReason'].agg(lambda x: ','.join(x.dropna())).reset_index()

In [9]:
final_DF.columns

Index(['Machine_Type', 'Machine', 'GrownWeight', 'ProdHeight', 'Dmtr',
       'Actual_Hours', 'GrRate', 'Stop_Reasons', 'LateStartReason'],
      dtype='object')

In [10]:
df_Stop_reason['Stop_Reasons_counts'] = df_Stop_reason['Stop_Reasons']
df_Stop_reason = count_words(df_Stop_reason,'Stop_Reasons_counts')

df_LateStartReason['LateStartReason_counts'] = df_LateStartReason['LateStartReason']
df_LateStartReason = count_words(df_LateStartReason,'LateStartReason_counts')

In [11]:
final_DF.GrownWeight = pd.to_numeric(final_DF.GrownWeight)
final_DF.ProdHeight = pd.to_numeric(final_DF.ProdHeight)
final_DF.Dmtr = pd.to_numeric(final_DF.Dmtr)
final_DF.Actual_Hours = pd.to_numeric(final_DF.Actual_Hours)
final_DF.GrRate = pd.to_numeric(final_DF.GrRate)

In [12]:
df_machine_less35 = pd.merge(
        final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight'].sum().reset_index(),
        final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(), 
                                      how="left", on=["Machine", "Machine_Type"])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10988\700123846.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(),


In [13]:
df_machine_less35.head(2)

,Machine,Machine_Type,GrownWeight,GrRate,Actual_Hours,ProdHeight,Dmtr
0,AA1,Old,264.03016,12.855,136.533333,3.069045,41.901667
1,AA10,Old,272.45215,11.910,140.900000,3.240711,44.670000


In [14]:
df_machine_less35.index.name = "ID"
df_machine_less35.reset_index(inplace=True)

In [15]:
df_machine_less35.head(2)

,ID,Machine,Machine_Type,GrownWeight,GrRate,Actual_Hours,ProdHeight,Dmtr
0,0,AA1,Old,264.03016,12.855,136.533333,3.069045,41.901667
1,1,AA10,Old,272.45215,11.910,140.900000,3.240711,44.670000


In [16]:
df_machine_less35["GrownWeight_less_35"] = df_machine_less35['GrownWeight'] - (df_machine_less35["GrownWeight"] * 35 / 100)
df_machine_less35["GrRate_rank"] = df_machine_less35['GrRate'].apply(lambda x: 1 if (x>12.50 and x<=13.00 ) else 0)

In [18]:
def add_sorting_rank_to_column(temp_df, column_name, ascending_order):
    temp_df[column_name] = pd.to_numeric(temp_df[column_name])
    temp_df = temp_df.sort_values(by=[column_name], ascending=ascending_order).reset_index()
    temp_df.index.name = column_name + "_rank"
    temp_df = temp_df.reset_index()
    temp_df[column_name + "_rank"] += 1
    temp_df.index.name = "delete_" + column_name
    return temp_df

In [19]:
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrownWeight", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Actual_Hours", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "ProdHeight", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Dmtr", False)
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35 = df_machine_less35.drop(["delete_Dmtr","delete_ProdHeight","delete_Actual_Hours","delete_GrownWeight","index"], axis=1)

In [20]:
df_machine_less35['Total_rank'] = df_machine_less35['GrownWeight_rank'] + df_machine_less35['Dmtr_rank'] + \
                                        df_machine_less35['Actual_Hours_rank'] + df_machine_less35["ProdHeight_rank"]

In [21]:
df_results = pd.merge(pd.merge(df_machine_less35,df_Stop_reason,how="left", on=["Machine", "Machine_Type"]),df_LateStartReason,how="left", on=["Machine", "Machine_Type"])

In [22]:
df_results = df_results[['ID','Machine_Type','Machine','GrownWeight','GrownWeight_less_35','Dmtr','Actual_Hours', 
                         'ProdHeight','GrRate','GrownWeight_rank','Actual_Hours_rank', 'ProdHeight_rank', 'Dmtr_rank',
                         'GrRate_rank','Total_rank','Stop_Reasons_counts', 'LateStartReason_counts']]

In [23]:
df_results = df_results.sort_values(by=['ID'])
df_results = df_results.set_index("ID")
if df_results.index[0] == 0:
    df_results.index += 1

In [24]:
if os.path.exists("final.xlsx"):
    os.remove("final.xlsx")
    
df_results.to_excel("final.xlsx", header=1, index_label="ID")

In [25]:
m,t = 'AC61','Old'
final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)]

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
774,Old,AC61,35.997415,1.668732,47.67,206.0,11.75,"TIDE O , N/A",ON TIME
775,Old,AC61,65.775000,3.065000,41.96,200.0,11.95,"TIDE O , N/A",ON TIME
776,Old,AC61,52.785000,2.777500,38.92,180.0,12.56,"TIDE O , N/A",ON TIME
777,Old,AC61,56.355000,4.095000,41.13,180.0,11.89,"TIDE O , N/A",POWER PROBLEM
778,Old,AC61,9.887940,0.412898,39.59,270.0,12.37,"TIDE O , N/A",MW


In [26]:
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrownWeight.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Actual_Hours.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].ProdHeight.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Dmtr.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrRate.mean())

220.80035499999997
207.2
2.4038259999999996
41.854
12.104
